<a href="https://colab.research.google.com/github/minassy/ISID_2021/blob/main/210916_%E3%83%A2%E3%83%87%E3%83%AB%E6%A4%9C%E8%A8%8E_AE_%E3%83%9A%E3%83%B3%E3%83%87%E3%82%A3%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import glob
import pandas as pd
import numpy as np
import pathlib
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#作業基本ディレクトリ
DIR = 'drive/MyDrive/00_Competition/ISID/2021'
os.listdir(DIR)
#os.mkdir(DIR+'/data')

['data',
 '210904_学習データの読み込み.ipynb',
 '210906_EDA_train.ipynb',
 '210908_EDA_test.ipynb',
 '210908_テストデータの読み込み_作成.ipynb',
 '210909_特徴量エンジニアリング.ipynb',
 '210912_特徴量E_連続処理_モデル検討_OneClassSVM.ipynb',
 '210916_モデル検討_AE.ipynb']

In [3]:
DIR_DATA_TRAIN = os.path.join(DIR, 'data', 'Train_Unit_20210903')
DIR_DATA_TEST = os.path.join(DIR, 'data', 'Test_Unit_20210903')

データ処理

In [4]:
#一連のデータ処理
def get_some_values(file_path):
  #ファイルパスから、ユニット名を取得。
  dirname = os.path.dirname(file_path)
  dirname_split = dirname.split('/')#区切り文字'/'で分割
  unit = dirname_split[-2]
  charge_mode = dirname_split[-1]
  #ファイルパスから、サイクル数を取得。
  basename = os.path.basename(file_path)
  basename_split = basename.split('_')[4]#区切り文字'_'で分割
  cycle_num = basename_split.split('.')[0]
  return unit, charge_mode, cycle_num

In [25]:
#%time
#DATA_DIR = DIR_DATA_TRAIN
#DATA_DIR = DIR_DATA_TEST

#C = 'Charge'
#C = 'Discharge'

def data_reading(DATA_DIR, C):

  #カラム名変更前後の辞書
  col_list = {'Time (s)' : 'Time',
              'Voltage Measured (V)' : 'VM',
              'Voltage Charge (V)' : 'VC',
              'Current Measured (Amps)' : 'CM', 
              'Current Charge (Amps)' : 'CC',
              'Temperature (degree C)': 'Temp', 
              'unit' : 'unit_name', 
              'charge_mode' : 'mode', 
              'Cycle_num' : 'Cycle',
              'Voltage Load (V)' : 'VL',
              'Current Load (Amps)' : 'CL'
  }

  df_list = []
  for folder in tqdm(os.listdir(DATA_DIR)[:3]):
    files = glob.glob(os.path.join(DATA_DIR, folder, C, '*.csv'))
    df_list_0 = []
    for file in files:
      tmp_df = pd.read_csv(file, encoding='utf-8')
      tmp_df = tmp_df.rename(columns=col_list)
      #ユニット名、充・放電モード、サイクル数の取得
      unit, charge_mode, cycle_num = get_some_values(file)
      #特徴量のデータフレームの作成(充電)
      df_exp = pd.DataFrame([{'unit':unit,
                        'charge_mode' : charge_mode,
                        'Cycle_num' : int(cycle_num),
                        'feature_c_vm' : featured_c_vm(tmp_df),
                        'feature_c_cm' : featured_c_cm(tmp_df),
                        'feature_c_vc' : featured_c_vc(tmp_df)}])
      df_list_0.append(df_exp)
    df_0 = pd.concat(df_list_0)
    df_list.append(df_0)
  df = pd.concat(df_list)

  #ユニットとサイクルでソート。
  df = df.sort_values(['unit', 'Cycle_num'])
  #インデックスの振り直し
  df = df.reset_index(drop=True)

  return df

特徴量化

In [16]:
def featured_c_vm(df):
  #LAG=5が0.001未満が続くところを、飽和領域とみなす。
  p_c_vm = df.loc[:, 'VM'][df.loc[:, 'VM'].diff(5) < 0.001]
  #飽和領域のなかで、最初の点を点P_C_VMとする。
  p_c_vm = p_c_vm.reset_index(inplace=False)
  p_c_vm = p_c_vm['index'].iloc[0]
  #充電VMの特徴量は、点P_C_VMにおける時間
  feature_c_vm = df['Time'].loc[p_c_vm]
  return feature_c_vm

def featured_c_cm(df):
  #飽和領域からの立ち下がり点(LAG=5の勾配が最小となる点)を点P_C_CMとする。
  p_c_cm = np.gradient(df.loc[:, 'CM'].diff(5)[10:]).argmin()
  #充電CMの特徴量は、点P_C_CMにおける時間
  feature_c_cm = df['Time'].loc[p_c_cm]
  return feature_c_cm

def featured_c_vc(df):
  #VCが最大値を示す点を、点P_C_VCとする。
  p_c_vc = df.VC.argmax()
  #充電VCの特徴量は、点P_C_VCにおける時間
  feature_c_vc = df['Time'].loc[p_c_vc]
  return feature_c_vc

def featured_d_vm(df):
  #VMが最小値を示す点を、点P_D_VMとする。
  p_d_vm = df.VM.argmin()
  #放電VMの特徴量は、点P_D_VMにおける時間
  feature_d_vm = df['Time'].loc[p_d_vm]
  return feature_d_vm

def featured_d_vl(df):
  #VLの最大値点以降の領域において、最小値を示す点を、点P_D_VLとする。
  VL_max = df.VL.argmax()
  p_d_vl = df.VL[VL_max: ].argmin()
  #放電VLの特徴量は、点P_D_VLにおける時間
  feature_d_vl = df['Time'].loc[p_d_vl]
  return feature_d_vl

In [26]:
%time
#データの読み込み(学習)

DATA_DIR = DIR_DATA_TRAIN
#DATA_DIR = DIR_DATA_TEST

C = 'Charge'
#C = 'Discharge'

df_train = data_reading(DATA_DIR, C)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.54 µs


100%|██████████| 3/3 [00:02<00:00,  1.06it/s]


In [27]:
df_train.head()

,unit,charge_mode,Cycle_num,feature_c_vm,feature_c_cm,feature_c_vc
0,Train_Unit_1,Charge,1,3422.5,3207.4,3241.8
1,Train_Unit_1,Charge,2,3431.6,3181.8,3262.3
2,Train_Unit_1,Charge,3,3409.1,9692.7,3229.2
3,Train_Unit_1,Charge,4,3407.7,3160.2,3228.2
4,Train_Unit_1,Charge,5,3409.6,3172.4,3217.9


In [28]:
%time
#データの読み込み(テスト)
#DATA_DIR = DIR_DATA_TRAIN
DATA_DIR = DIR_DATA_TEST
C = 'Charge'
#C = 'Discharge'

df_test = data_reading(DATA_DIR, C)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.68 µs


100%|██████████| 3/3 [00:36<00:00, 12.18s/it]


In [29]:
df_test.head()

,unit,charge_mode,Cycle_num,feature_c_vm,feature_c_cm,feature_c_vc
0,Test_Unit_1,Charge,1,741.86,3476.3,3141.4
1,Test_Unit_1,Charge,2,849.45,3451.6,3164.0
2,Test_Unit_1,Charge,3,803.84,8280.7,3145.7
3,Test_Unit_1,Charge,4,806.83,3102.9,3123.4
4,Test_Unit_1,Charge,5,1004.50,3147.9,3163.0


# モデル化

In [30]:
from sklearn import preprocessing
from sklearn.svm import OneClassSVM

import keras.models
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.models import Sequential

入力サイズ→64次元→8次元と圧縮した後、8次元→64次元→出力サイズ(=入力サイズ)の順に復元するAEを作成。

In [31]:
inputDim = 40064
model = Sequential()

In [ ]:
model.add(Dense(64, input_shape=(inputDim,)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(8))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))